In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from tabulate import tabulate
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/03&04_17_VSE.csv'
df_orig = pd.read_csv(file_path, encoding='cp1251', sep=";", thousands=' ', decimal=',')


In [4]:
df = df_orig
df.info()

In [5]:
df

In [6]:
def columnValues(df):
    row = [df.nunique(), (df == 0).sum(axis=0), df.isna().sum(), round(df.isna().sum() / len(df) * 100, 1), df.dtypes]
    return row

data = []
for column in df:
    data.append([column] + columnValues(df[column]))

print(tabulate(data, headers=['Column', 'Count Unique', 'Count Zeros', 'Count NaNs','% of NaNs', 'data type'], tablefmt='orgtbl'))

In [7]:
df['НовыйСтатус'].value_counts()

In [8]:
DROP_ALWAYS = [
    'ID_SKU', 'ЭлектроннаяПочта_new', 'Клиент', 'ПВЗ_код',
    'ГородМагазина', 'МагазинЗаказа', 'Маржа', 'ЦенаЗакупки',
    'НомерСтроки', 'OneClick',           # тех-ID
    'МетодДоставки',                     # удаляем delivery-фичи
    'Дата', 'ДатаДоставки', 'ДатаЗаказаНаСайте',
    'МесяцДатыЗаказа', 'ГодДатыЗаказа',  # календарь не нужен
    'Статус'                             # дублирует НовыйСтатус
]
df.drop(columns=[c for c in DROP_ALWAYS if c in df.columns], inplace=True)

In [9]:
# ───────────────────── 0. Справочники ──────────────────────────────
SUBJECT2FO = {
    # Центральный (ЦФО)
    'Москва':'ЦФО','Московская область':'ЦФО','Тверская область':'ЦФО','Смоленская область':'ЦФО',
    'Тульская область':'ЦФО','Калужская область':'ЦФО','Ярославская область':'ЦФО',
    'Владимирская область':'ЦФО','Рязанская область':'ЦФО','Ивановская область':'ЦФО',
    'Костромская область':'ЦФО','Белгородская область':'ЦФО','Курская область':'ЦФО',
    'Орловская область':'ЦФО','Брянская область':'ЦФО','Липецкая область':'ЦФО',
    'Тамбовская область':'ЦФО','Воронежская область':'ЦФО',
    # Северо-Запад
    'Санкт-Петербург':'СЗФО','Ленинградская область':'СЗФО','Новгородская область':'СЗФО',
    'Псковская область':'СЗФО','Архангельская область':'СЗФО','Мурманская область':'СЗФО',
    'Вологодская область':'СЗФО','Карелия':'СЗФО','Коми':'СЗФО','Ненецкий автономный округ':'СЗФО',
    # Южный
    'Краснодарский край':'ЮФО','Республика Крым':'ЮФО','Севастополь':'ЮФО',
    'Ростовская область':'ЮФО','Волгоградская область':'ЮФО','Астраханская область':'ЮФО',
    'Калмыкия':'ЮФО','Адыгея':'ЮФО',
    # Северо-Кавказский
    'Ставропольский край':'СКФО','Дагестан':'СКФО','Чечня':'СКФО','Ингушетия':'СКФО',
    'Северная Осетия':'СКФО','Кабардино-Балкария':'СКФО','Карачаево-Черкесия':'СКФО',
    # Приволжский
    'Нижегородская область':'ПФО','Пермский край':'ПФО','Татарстан':'ПФО','Башкортостан':'ПФО',
    'Удмуртия':'ПФО','Чувашия':'ПФО','Марий Эл':'ПФО','Мордовия':'ПФО',
    'Оренбургская область':'ПФО','Самарская область':'ПФО','Саратовская область':'ПФО',
    'Ульяновская область':'ПФО','Пензенская область':'ПФО','Кировская область':'ПФО',
    # Уральский
    'Свердловская область':'УФО','Челябинская область':'УФО','Тюменская область':'УФО',
    'Курганская область':'УФО','Ханты-Мансийский автономный округ':'УФО',
    'Ямало-Ненецкий автономный округ':'УФО',
    # Сибирский
    'Алтайский край':'СФО','Красноярский край':'СФО','Кемеровская область':'СФО',
    'Новосибирская область':'СФО','Омская область':'СФО','Томская область':'СФО',
    'Иркутская область':'СФО','Бурятия':'СФО','Тыва':'СФО','Хакасия':'СФО',
    'Забайкальский край':'СФО',
    # Дальневосточный
    'Хабаровский край':'ДФО','Приморский край':'ДФО','Амурская область':'ДФО',
    'Сахалинская область':'ДФО','Магаданская область':'ДФО','Камчатский край':'ДФО',
    'Чукотский автономный округ':'ДФО','Якутия':'ДФО','Еврейская автономная область':'ДФО',
}

# (1) «Критические» города или посёлки, где субъект не указан в скобках
CITY2SUBJECT = {
    'Москва':'Москва','Зеленоград':'Москва','Подольск':'Московская область','Королев':'Московская область',
    'Химки':'Московская область','Балашиха':'Московская область','Мытищи':'Московская область',
    'Люберцы':'Московская область','Красногорск':'Московская область','Электросталь':'Московская область',
    'Челябинск':'Челябинская область','Екатеринбург':'Свердловская область','Нижний Новгород':'Нижегородская область',
    'Казань':'Татарстан','Самара':'Самарская область','Уфа':'Башкортостан','Пермь':'Пермский край',
    'Ростов-на-Дону':'Ростовская область','Краснодар':'Краснодарский край','Сочи':'Краснодарский край',
    'Волгоград':'Волгоградская область','Астрахань':'Астраханская область','Саратов':'Саратовская область',
    'Ижевск':'Удмуртия','Йошкар-Ола':'Марий Эл','Чебоксары':'Чувашия','Тверь':'Тверская область',
    'Тула':'Тульская область','Брянск':'Брянская область','Смоленск':'Смоленская область',
    'Воронеж':'Воронежская область','Белгород':'Белгородская область','Курск':'Курская область',
    'Калуга':'Калужская область','Иваново':'Ивановская область','Ярославль':'Ярославская область',
    'Вологда':'Вологодская область','Архангельск':'Архангельская область','Мурманск':'Мурманская область',
    'Петрозаводск':'Карелия','Сыктывкар':'Коми','Новосибирск':'Новосибирская область',
    'Омск':'Омская область','Томск':'Томская область','Барнаул':'Алтайский край',
    'Кемерово':'Кемеровская область','Красноярск':'Красноярский край','Иркутск':'Иркутская область',
    'Чита':'Забайкальский край','Хабаровск':'Хабаровский край','Владивосток':'Приморский край',
    'Южно-Сахалинск':'Сахалинская область','Магадан':'Магаданская область','Якутск':'Якутия',
    'Петропавловск-Камчатский':'Камчатский край','Симферополь':'Республика Крым','Севастополь':'Севастополь',
    'Грозный':'Чечня','Махачкала':'Дагестан','Назрань':'Ингушетия','Нальчик':'Кабардино-Балкария',
    'Владикавказ':'Северная Осетия',
}

import re, pandas as pd

def map_fo(region: str) -> str:
    """Определяет федеральный округ по строке `region`."""
    if pd.isna(region):
        return 'Other_FO'

    txt = str(region).strip()

    # 1) если в скобках указан субъект — берём его
    match = re.search(r'\(([^)]+)\)', txt)
    if match:
        subj = match.group(1).replace('район','').strip()
        if subj in SUBJECT2FO:
            return SUBJECT2FO[subj]

    # 2) прямой словарь “город → субъект”
    if txt in CITY2SUBJECT:
        subj = CITY2SUBJECT[txt]
        return SUBJECT2FO.get(subj, 'Other_FO')

    # 3) ищем любое вхождение субъекта в строке
    for subj, fo in SUBJECT2FO.items():
        if subj in txt:
            return fo

    # 4) не нашли
    return 'Other_FO'


In [10]:
df = df[df['Отменено'] != 'Да']                          # только завершённые
df = df[df['КоличествоПроданоКлиенту'] > 0]              # нет частичных возвратов
df = df[~df['ПричинаОтмены'].str.contains('тест|дубл', case=False, na=False)]
df.drop(columns=['Отменено', 'ПричинаОтмены'], inplace=True, errors='ignore')

df = df[df['Телефон_new'].notna() & (df['Телефон_new'] != '0')]  # валидный ID
df = df[df['НовыйСтатус'] == 'Доставлен']                      # оставляем только доставленные
df = df[df['Тип'].str.lower() != 'иное']                         # убираем «иное»
df = df[df['Группа2'].notna()]                                   # нужная категория
# df = df[(df['Цена'] > 0) & (df['СуммаСтроки'] > 0) & (df['Цена'] >= 200)]

In [11]:
data = []
for column in df:
    data.append([column] + columnValues(df[column]))

print(tabulate(data, headers=['Column', 'Count Unique', 'Count Zeros', 'Count NaNs','% of NaNs', 'data type'], tablefmt='orgtbl'))

In [12]:
df.dropna(subset=['Регион', 'Группа4'], inplace=True)

In [13]:
df['СуммаБезСкидки'] = df['Цена'] * df['Количество']
df['Скидка'] = df['СуммаБезСкидки'] - df['СуммаСтроки']
# def map_fo(txt):
#     if pd.isna(txt): return 'Other_FO'
#     m = re.search(r'\(([^)]+)\)', str(txt))
#     if m and m.group(1).strip() in SUBJECT2FO:
#         return SUBJECT2FO[m.group(1).strip()]
#     if txt in CITY2SUBJECT:
#         return SUBJECT2FO.get(CITY2SUBJECT[txt], 'Other_FO')
#     for sbj, fo in SUBJECT2FO.items():
#         if sbj in txt: return fo
#     return 'Other_FO'

# df['ФО'] = df['Регион'].apply(map_fo)


In [14]:
id_col = 'Телефон_new'
agg = (df.groupby(id_col)
         .agg(ЧислоЗаказов=('НомерЗаказаНаСайте', 'nunique'),
              ОбщаяСуммаПокупок=('СуммаСтроки', 'sum'),
              ОбщееКоличествоТоваров=('Количество', 'sum'),
              СуммаСкидки=('Скидка', 'sum'))
      )

# производные метрики
agg['СреднийЧек']            = (agg['ОбщаяСуммаПокупок'] / agg['ЧислоЗаказов']).round(2)
agg['СрКолТоваров_вЧеке']    = (agg['ОбщееКоличествоТоваров'] / agg['ЧислоЗаказов']).round(2)
agg['ДоляСкидки']            = (agg['СуммаСкидки'] / agg['ОбщаяСуммаПокупок']).round(3)

In [15]:
def ohe(col, pref):
    return (pd.crosstab(df[id_col], df[col])
              .add_prefix(pref)
              .astype('int16'))

vit = pd.concat([
        agg,
        ohe('Группа2',     'G2_'),      # 13 категорий товаров
        # ohe('ФормаОплаты', 'Pay_'),     # 2 способа оплаты
        # ohe('ФО',          'FO_')       # 8 округов + Other
      ], axis=1).fillna(0)

vit = vit.copy()   # дефрагментация
print('финальная витрина:', vit.shape)
vit.head()

In [16]:
vit

In [17]:
vit.info()

In [20]:
vit.describe()

In [19]:
vit.drop(columns=['СуммаСкидки', 'ДоляСкидки'], inplace=True)

In [21]:
vit.to_csv('vitrina_clean.csv', index=True, encoding='utf8')

In [22]:
from sklearn.preprocessing import StandardScaler
import numpy as np



OUTLIER_SHARE   = 0.10     # 1 % клиентов считаем выбросами
# FO_WEIGHT       = 0.3      # ослабляем регионы
RANDOM_STATE    = 42
K_RANGE         = range(2, 11)   # перебираем k = 2 … 10


num_cols = ['ЧислоЗаказов', 'ОбщаяСуммаПокупок', 'ОбщееКоличествоТоваров',
             'СреднийЧек',        'СрКолТоваров_вЧеке']
# fo_cols  = [c for c in vit.columns if c.startswith('FO_')]
bin_cols = vit.columns.difference(num_cols)

X = np.hstack([
        StandardScaler().fit_transform(vit[num_cols]),       # 7 числовых
        vit[bin_cols].values.astype('float32'),              #  (13+2) бинарных
        # vit[fo_cols].values.astype('float32') * FO_WEIGHT    # 8–9 FO_ ×0.3
    ])

In [23]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(contamination=OUTLIER_SHARE, n_estimators=200,
                      random_state=RANDOM_STATE, n_jobs=-1).fit(X)
mask_ok = iso.predict(X) == 1
print('выбросов удалено:', (~mask_ok).sum())

vit_ok = vit.loc[mask_ok].copy()
X_ok   = X[mask_ok]

In [24]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics  import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA


K_RANGE = range(2,15)
RND = 42

sil, ch, db, sse = [], [], [], []
sample = np.random.default_rng(RND).choice(X_ok.shape[0], 20000, replace=False)

print("k | sil |   CH   |  DB  |   SSE ")
for k in K_RANGE:
    km = KMeans(k, n_init=30, max_iter=600, random_state=RND).fit(X_ok)
    lbl = km.labels_
    sil.append(silhouette_score(X_ok[sample], lbl[sample]))
    ch.append(calinski_harabasz_score(X_ok, lbl))
    db.append(davies_bouldin_score(X_ok, lbl))
    sse.append(km.inertia_)
    print(f"{k:2d}|{sil[-1]:.3f}|{ch[-1]:7.0f}|{db[-1]:5.2f}|{sse[-1]:8.0f}")

fig, ax = plt.subplots(2, 2, figsize=(12, 8))

ax[0, 0].plot(K_RANGE, sil, marker='o')
ax[0, 0].set_title('Silhouette (↑)')
ax[0, 0].set_xlabel('k')
ax[0, 0].set_ylabel('score')

ax[0, 1].plot(K_RANGE, sse, marker='o')
ax[0, 1].set_title('Elbow (SSE ↓)')
ax[0, 1].set_xlabel('k')
ax[0, 1].set_ylabel('SSE')

ax[1, 0].plot(K_RANGE, ch, marker='o')
ax[1, 0].set_title('Calinski–Harabasz (↑)')
ax[1, 0].set_xlabel('k')
ax[1, 0].set_ylabel('score')

ax[1, 1].plot(K_RANGE, db, marker='o')
ax[1, 1].set_title('Davies–Bouldin (↓)')
ax[1, 1].set_xlabel('k')
ax[1, 1].set_ylabel('score')

plt.tight_layout()
plt.show()


In [25]:
RND = 42
KS  = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] # № кластеров
SAMPLE = 20000            # сколько точек на scatter
pca = PCA(n_components=2, random_state=RND)
coords = pca.fit_transform(X_ok)

rng   = np.random.default_rng(RND)
idx   = rng.choice(len(coords), SAMPLE, replace=False)
coords_s = coords[idx]

# --- готовим фигуру ------------------------------------------------
fig, axes = plt.subplots(len(KS), 2, figsize=(12, 4*len(KS)))

for row, k in enumerate(KS):
    km  = KMeans(k, n_init=30, max_iter=600, random_state=RND).fit(X_ok)
    labs_full = km.labels_
    labs_s    = labs_full[idx]

    # ---- scatter PCA ----
    ax_sc = axes[row, 0]
    sc = ax_sc.scatter(coords_s[:,0], coords_s[:,1],
                       c=labs_s, s=6, alpha=0.6, cmap='tab10')
    ax_sc.set_title(f"PCA (k={k}) — PC1={pca.explained_variance_ratio_[0]*100:.1f}%, "
                    f"PC2={pca.explained_variance_ratio_[1]*100:.1f}%")
    ax_sc.set_xlabel("PC1"); ax_sc.set_ylabel("PC2")
    ax_sc.grid(ls='--', alpha=0.3)

    # ---- pie chart ----
    ax_pie = axes[row, 1]
    counts = np.bincount(labs_full)
    labels = [str(i) for i in range(k)]
    ax_pie.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90,
               colors=plt.cm.tab10.colors)
    ax_pie.set_title(f"Распределение клиентов (k={k})")

plt.tight_layout()
plt.show()

In [ ]:
# vit_ok['Кластер'] = KMeans(6, n_init=40, max_iter=700,
#                            random_state=42).fit_predict(X_ok)


In [26]:
from sklearn.mixture import GaussianMixture


RND = 42
COMPONENTS = range(2, 15)

sil, ch, db, bic, aic = [], [], [], [], []

sample = np.random.default_rng(RND).choice(X_ok.shape[0], 20000, replace=False)

print("k | sil |  CH  |  DB  |   BIC   |   AIC")
for k in COMPONENTS:
    gmm = GaussianMixture(n_components=k, covariance_type='diag',
                          random_state=RND, n_init=5).fit(X_ok)
    labels = gmm.predict(X_ok)
    sil.append(silhouette_score(X_ok[sample], labels[sample]))
    ch.append(calinski_harabasz_score(X_ok, labels))
    db.append(davies_bouldin_score(X_ok, labels))
    bic.append(gmm.bic(X_ok))
    aic.append(gmm.aic(X_ok))
    print(f"{k:2d}|{sil[-1]:.3f}|{ch[-1]:6.0f}|{db[-1]:5.2f}|{bic[-1]:8.0f}|{aic[-1]:8.0f}")

# --- графики для GMM ---
fig, ax = plt.subplots(2, 3, figsize=(15, 8))

ax[0,0].plot(COMPONENTS, sil, 'o-'); ax[0,0].set_title('Silhouette (↑)'); ax[0,0].set_xlabel('k')
ax[0,1].plot(COMPONENTS, ch, 'o-');  ax[0,1].set_title('Calinski‑Harabasz (↑)'); ax[0,1].set_xlabel('k')
ax[0,2].plot(COMPONENTS, db, 'o-');  ax[0,2].set_title('Davies‑Bouldin (↓)');   ax[0,2].set_xlabel('k')

ax[1,0].plot(COMPONENTS, bic, 'o-'); ax[1,0].set_title('BIC (↓)'); ax[1,0].set_xlabel('k')
ax[1,1].plot(COMPONENTS, aic, 'o-'); ax[1,1].set_title('AIC (↓)'); ax[1,1].set_xlabel('k')

ax[1,2].axis('off')   # пустая ячейка для ровной сетки

plt.tight_layout()
plt.show()

In [ ]:
# BEST_K = 11
# gmm = GaussianMixture(n_components=BEST_K, covariance_type='diag',
#                       random_state=42, n_init=5).fit(X_ok)
# vit_ok['EM_Кластер'] = gmm.predict(X_ok)

# print('Размеры EM-кластеров:', np.bincount(vit_ok['EM_Кластер']))


In [27]:
# prompt: visualize pca client distribution among 10, 11, 12  gmm clusters in pie chart

RND = 42
KS_GMM = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] # № кластеров для GMM
SAMPLE = 20000            # сколько точек на scatter
pca = PCA(n_components=2, random_state=RND)
coords = pca.fit_transform(X_ok)

rng   = np.random.default_rng(RND)
idx   = rng.choice(len(coords), SAMPLE, replace=False)
coords_s = coords[idx]

# --- готовим фигуру для GMM ---
fig, axes = plt.subplots(len(KS_GMM), 2, figsize=(12, 4*len(KS_GMM)))

for row, k in enumerate(KS_GMM):
    gmm = GaussianMixture(n_components=k, covariance_type='diag',
                          random_state=RND, n_init=5).fit(X_ok)
    labs_full = gmm.predict(X_ok)
    labs_s    = labs_full[idx]

    # ---- scatter PCA ----
    ax_sc = axes[row, 0]
    sc = ax_sc.scatter(coords_s[:,0], coords_s[:,1],
                       c=labs_s, s=6, alpha=0.6, cmap='tab10')
    ax_sc.set_title(f"PCA (GMM, k={k}) — PC1={pca.explained_variance_ratio_[0]*100:.1f}%, "
                    f"PC2={pca.explained_variance_ratio_[1]*100:.1f}%")
    ax_sc.set_xlabel("PC1"); ax_sc.set_ylabel("PC2")
    ax_sc.grid(ls='--', alpha=0.3)

    # ---- pie chart ----
    ax_pie = axes[row, 1]
    counts = np.bincount(labs_full)
    labels = [str(i) for i in range(k)]
    ax_pie.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90,
               colors=plt.cm.tab20.colors) # Using tab20 as tab10 might not have enough colors for 12 clusters
    ax_pie.set_title(f"Распределение клиентов (GMM, k={k})")

plt.tight_layout()
plt.show()


In [28]:
vit_ok

In [29]:
k_list = [9, 10, 11]
gmm_list = [8, 9, 10, 11]

# ────────── K-means (добавим, если ещё нет) ──────────
for k in k_list:
    col = f'K{k}_Кластер'
    if col not in vit_ok.columns:
        vit_ok[col] = KMeans(k, n_init=40, max_iter=700,
                             random_state=RND).fit_predict(X_ok)
        print(f'K-means k={k} → добавлен столбец {col}')

# ────────── GMM / EM (добавим, если ещё нет) ─────────
for k in gmm_list:
    col = f'G{k}_Кластер'
    if col not in vit_ok.columns:
        gmm = GaussianMixture(n_components=k,
                              covariance_type='full',
                              init_params='kmeans',
                              n_init=10,
                              random_state=RND).fit(X_ok)
        vit_ok[col] = gmm.predict(X_ok)
        print(f'GMM k={k} → добавлен столбец {col}')


In [30]:
vit_ok

In [31]:
# # список «устаревших» колонок — удаляем, если есть
# drop_cols = ['Кластер', 'EM_Кластер']
# vit_ok = vit_ok.drop(columns=[c for c in drop_cols if c in vit_ok.columns])

# сохраняем обновлённую версию
vit_ok.to_csv('vitrina_clients_clusters_all_new.csv', index=True, encoding='utf-8')
print('файл перезаписан.')


In [32]:
vit_ok

In [33]:
df

In [34]:
cluster_cols = ['K9_Кластер','K10_Кластер','K11_Кластер',
                'G8_Кластер','G9_Кластер','G10_Кластер', 'G11_Кластер']

df_full = df.merge(
            vit_ok[cluster_cols],
            left_on='Телефон_new',     # ключ в df
            right_index=True,          # индекс = Телефон_new в vit_ok
            how='left'
          )

print('Размер после merge:', df_full.shape)


In [35]:
#  CSV сжатый (совместим с Excel 2016+)
csv_name = 'data/transactions_clean_with_clusters.csv'
df_full.to_csv(csv_name, index=False, encoding='utf-8')




In [36]:
df.describe()

In [37]:
df_final = pd.read_csv('data/transactions_clean_with_clusters.csv', encoding='utf-8')
df_final